# Data Prepatation



In [86]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import json
from config import CLIENT_ID, CLIENT_SECRET


### Download and preprocess data

In [87]:
def clean_sheets(sheets):
    '''
    Get rid of the duplicate sheets
    Only take the sheets from the latest point in the day
    '''
    
    new_ranges = []
    for s in sheets:
        new_ranges.append(s.get("properties", {}).get("title"))
        
    clean_new_ranges = new_ranges.copy()
    for i, x in enumerate(clean_new_ranges):
        clean_new_ranges[i] = x.split('_')[0]

    indices = []
    for item in set(clean_new_ranges):
        indices.append(clean_new_ranges.index(item))

    clean_new_ranges = []
    for index in sorted(indices):
        clean_new_ranges.append(new_ranges[index])

    print(clean_new_ranges)
    return clean_new_ranges
    

In [88]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w'

"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

#get all the sheet names for ranges when querying
sheet_metadata = service.spreadsheets().get(spreadsheetId=SAMPLE_SPREADSHEET_ID).execute()
sheets = sheet_metadata.get('sheets', '')

cleaned_ranges = clean_sheets(sheets)

['Jan29_230pm', 'Jan28_11pm', 'Jan27_830pm', 'Jan26_11pm', 'Jan25_10pm', 'Jan24_12pm', 'Jan23_12pm', 'Jan22_12pm']


In [114]:
'''
Now that we have the sheet names cleaned
we can now start doing some extractions
'''

# df = pd.DataFrame()

def get_col_names():
    col_names= []
    for sheet_range in cleaned_ranges:
        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=sheet_range).execute()
        values = result.get('values', [])      
        for col in values[0]:
            col_names.append(col[:])
    return set(col_names)

cols = get_col_names()
cols

df = pd.DataFrame(columns=cols)


In [131]:
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='Jan29_230pm').execute()

In [143]:
# print(json.dumps(result.get('values', []), indent=4, sort_keys=True))

In [140]:
print(json.dumps(result.get('values', [])[0][1], indent=4, sort_keys=True))

"Country/Region"


In [137]:
df

,Demised,Country/Region,Confirmed,Recovered,Country,Province/State,Deaths,Last Update,Suspected


In [163]:
def get_data():
    for sheet_range in cleaned_ranges:
        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range='Jan29_230pm').execute()
    
    for i, sheet_col in enumerate(result.get('values', [])[0]):
        print(sheet_col)
        
        print('    ', result.get('values', [])[1][i])
    
#         df[sheet_col] = result.get('values', [])[1][i]
        
    return df
    
df = get_data()
df

Province/State
     Hubei


TypeError: cannot concatenate object of type "<class 'str'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid